# DECP decp-json

update our datasets about "commande publique" on data.gouv.fr w/ data from Colin's decp-json

In [ ]:
import requests
from dateutil.parser import parse
from datetime import datetime

In [ ]:
PROD = False
BASE_URL = 'https://next.data.gouv.fr' if not PROD else 'https://data.gouv.fr'

## pes marché

In [ ]:
with open('datagouvfr-token.txt', 'r') as f:
    TOKEN = f.read()
DATASET_ID = '5bd0b6fd8b4c413d0801dc57'
API_URL = '%s/api/1/datasets/%s' % (BASE_URL, DATASET_ID)

In [ ]:
# create or update resource on data.gouv.fr

r = requests.get(API_URL)
resources = r.json()['resources']

json_r = [r for r in resources if r['format'] == 'json']
json_r = json_r[0] if json_r else None
if not json_r:
    R_API_URL = '%s/upload/' % (API_URL)
else:
    R_API_URL = '%s/resources/%s/' % (API_URL, json_r['id'])
    
last_modified_dest = parse(json_r['last_modified']) if json_r else None
last_modified_dest

In [ ]:
# get data from colin

SOURCE_URL = 'https://sireneld.io/data/decp/data.gouv.fr_pes.json'

r = requests.head(SOURCE_URL)
# cast to naive datetime
last_modified_source = parse(r.headers['Last-Modified']).replace(tzinfo=None)

if not json_r or (last_modified_dest < last_modified_source):
    source = requests.get(SOURCE_URL)
    files = {'file': ('decp-pes-%s.json' % last_modified_source.strftime('%Y-%m-%d'), source.content)}
    r = requests.post(R_API_URL, files=files, headers={'x-api-key': TOKEN})
    print('Updated PES json resource.')
else:
    print('Nothing to do.')